# Обучение модели Restormer для восстановления изображений с эффектом "плохой печати"
Этот Notebook демонстрирует процесс fine-tuning модели Restormer на датасете с искаженными изображениями (эффект плохой печати). 

Мы используем предобученные веса для denoising и адаптируем модель под разрешение 1024x1448. 

Обучение включает мониторинг метрик (Loss, PSNR, SSIM), early stopping и сохранение чекпоинтов.

## Шаг 1: Настройка окружения и выбор GPU
Выбираем свободную GPU, настраиваем PyTorch для оптимальной работы с CUDA.

In [ ]:
# Импорты и настройка
import os
import sys
import subprocess
import torch
from torch.amp import autocast, GradScaler
import torch.utils.checkpoint as checkpoint

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Получение списка GPU и выбор с максимум свободной памяти
result = subprocess.check_output(
    ["nvidia-smi", "--query-gpu=index,memory.free", "--format=csv,noheader,nounits"]
).decode().strip()

gpu_list = [tuple(map(int, line.split(', '))) for line in result.splitlines()]
best_idx, max_free = max(gpu_list, key=lambda x: x[1])

os.environ['CUDA_VISIBLE_DEVICES'] = str(best_idx)
print(f"Выбрана GPU {best_idx} — свободно ≈{max_free//1024} GiB")

# Установка устройства и очистка кэша
device = torch.device('cuda:0')
torch.cuda.set_device(device)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()

## Шаг 2: Загрузка модели Restormer
Импортируем модифицированную архитектуру Restormer с gradient checkpointing для экономии VRAM, загружаем предобученные веса.

In [ ]:
# Переход в директорию с архитектурой

from restormer_arch import Restormer
sys.path.append('/home/kudriavtcevroman-10')

# Инициализация модели с checkpointing
model = Restormer(
    inp_channels=3,
    out_channels=3,
    dim=48,
    num_blocks=[4, 6, 6, 8],
    num_refinement_blocks=4,
    heads=[1, 2, 4, 8],
    ffn_expansion_factor=2.66,
    bias=False,
    LayerNorm_type='BiasFree',
    dual_pixel_task=False,
    use_checkpoint=True
).to(device)

# Загрузка предобученных весов
weights_path = 'pretrained/gaussian_color_denoising_blind.pth'
checkpoint = torch.load(weights_path, map_location=device)
model.load_state_dict(checkpoint.get('params', checkpoint.get('state_dict', checkpoint)), strict=True)

print("Модель Restormer с gradient checkpointing успешно загружена")
print(f"VRAM после загрузки модели: {torch.cuda.memory_allocated(device)/1024**3:.1f} GiB")

## Шаг 3: Подготовка датасета
Определяем кастомный класс датасета для пар distorted/clean, с трансформацией на полное разрешение 1024x1448 и padding для совместимости с моделью.

In [ ]:
# Импорты для датасета
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

# Кастомный трансформер с padding
class PaddedResize:
    def __init__(self, size=(1024, 1448), div_factor=8):
        self.size = size
        self.div_factor = div_factor

    def __call__(self, img):
        img = transforms.Resize(self.size, interpolation=Image.LANCZOS)(img)
        w, h = img.size
        pad_w = (self.div_factor - w % self.div_factor) % self.div_factor
        pad_h = (self.div_factor - h % self.div_factor) % self.div_factor
        if pad_w != 0 or pad_h != 0:
            img = transforms.Pad((0, 0, pad_w, pad_h), fill=0)(img)
        return img

# Трансформации
transform = transforms.Compose([
    PaddedResize(size=(1024, 1448)),
    transforms.ToTensor(),
])

# Кастомный датасет
class BadPrintDataset(Dataset):
    def __init__(self, distorted_dir, clean_dir, transform=None):
        self.distorted_images = sorted(os.listdir(distorted_dir))
        self.clean_images = sorted(os.listdir(clean_dir))
        self.distorted_dir = distorted_dir
        self.clean_dir = clean_dir
        self.transform = transform

    def __len__(self): return len(self.distorted_images)

    def __getitem__(self, idx):
        dist = Image.open(os.path.join(self.distorted_dir, self.distorted_images[idx])).convert('RGB')
        clean = Image.open(os.path.join(self.clean_dir, self.clean_images[idx])).convert('RGB')
        if self.transform:
            dist = self.transform(dist)
            clean = self.transform(clean)
        return dist, clean

# Создание датасетов и лоадеров
train_dataset = BadPrintDataset('bad_print/train/distorted', 'bad_print/train/clean', transform=transform)
val_dataset = BadPrintDataset('bad_print/val/distorted', 'bad_print/val/clean', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

## Шаг 4: Настройка обучения
Определяем loss (Charbonnier), метрики (PSNR, SSIM), оптимизатор (AdamW), scheduler и early stopping.

In [ ]:
# Импорты для метрик и обучения
import torch.nn.functional as F
import math
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from tqdm import tqdm
import time

# Кастомный loss (Charbonnier)
def charbonnier_loss(pred, target, eps=1e-6):
    return torch.mean(torch.sqrt((pred - target) ** 2 + eps))

criterion = charbonnier_loss

# Метрики
psnr_metric = PeakSignalNoiseRatio(data_range=1.0).to(device)
ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)

# Оптимизатор и scaler
optimizer = AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
scaler = GradScaler()

# Scheduler
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

# Early stopping
patience = 10
early_stop_counter = 0
best_val_psnr = -float('inf')

# Директория для чекпоинтов
os.makedirs('checkpoints', exist_ok=True)

# Списки для метрик
train_losses, val_losses = [], []
train_psnrs, val_psnrs = [], []
train_ssims, val_ssims = [], []

num_epochs = 50

## Шаг 5: Цикл обучения
Обучаем модель с mixed precision, мониторим метрики, сохраняем чекпоинты и применяем early stopping.

In [ ]:
# Цикл обучения
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    train_loss = train_psnr = train_ssim = 0.0

    for dist, clean in tqdm(train_loader, desc=f"Epoch {epoch+1:02d}/{num_epochs} [train]"):
        dist = dist.to(device, non_blocking=True)
        clean = clean.to(device, non_blocking=True)
        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            pred = model(dist)
            loss = criterion(pred, clean)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
        with torch.no_grad():
            train_psnr += psnr_metric(pred, clean).item()
            train_ssim += ssim_metric(pred, clean).item()

    # Средние метрики train
    avg_train_loss = train_loss / len(train_loader)
    avg_train_psnr = train_psnr / len(train_loader)
    avg_train_ssim = train_ssim / len(train_loader)
    train_losses.append(avg_train_loss)
    train_psnrs.append(avg_train_psnr)
    train_ssims.append(avg_train_ssim)

    scheduler.step()

    # Валидация
    model.eval()
    val_loss = val_psnr = val_ssim = 0.0
    with torch.no_grad():
        for dist, clean in tqdm(val_loader, desc=f"Epoch {epoch+1:02d}/{num_epochs} [val]"):
            dist = dist.to(device, non_blocking=True)
            clean = clean.to(device, non_blocking=True)
            with autocast(device_type='cuda'):
                pred = model(dist)
                val_loss += criterion(pred, clean).item()
                val_psnr += psnr_metric(pred, clean).item()
                val_ssim += ssim_metric(pred, clean).item()

    # Средние метрики val
    avg_val_loss = val_loss / len(val_loader)
    avg_val_psnr = val_psnr / len(val_loader)
    avg_val_ssim = val_ssim / len(val_loader)
    val_losses.append(avg_val_loss)
    val_psnrs.append(avg_val_psnr)
    val_ssims.append(avg_val_ssim)

    print(f"\nЭпоха {epoch+1:02d} | Train Loss: {avg_train_loss:.5f} | Val Loss: {avg_val_loss:.5f} | "
          f"Val PSNR: {avg_val_psnr:.2f} dB | Val SSIM: {avg_val_ssim:.4f} | LR: {optimizer.param_groups[0]['lr']:.2e}")

    # Сохранение чекпоинта
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_psnr': best_val_psnr,
    }, f"checkpoints/restormer_badprint_ep{epoch+1}.pth")

    # Early stopping
    if avg_val_psnr > best_val_psnr:
        best_val_psnr = avg_val_psnr
        torch.save(model.state_dict(), "checkpoints/restormer_badprint_best_psnr.pth")
        print("    → Новая лучшая модель сохранена!")
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        print(f"    → Нет улучшения PSNR ({early_stop_counter}/{patience})")

    if early_stop_counter >= patience:
        print(f"Early stopping: переобучение обнаружено после {patience} эпох без улучшения.")
        break

    torch.cuda.empty_cache()

print("Fine-tuning Restormer завершён")

## Шаг 6: Сохранение финальной модели
Сохраняем обученные веса для дальнейшего использования в инференсе.

In [ ]:
# Сохранение финальной модели
torch.save(model.state_dict(), 'finetuned_restormer.pth')
print("Финальная модель сохранена как finetuned_restormer.pth.")

## Шаг 7: Визуализация метрик
Строим графики для анализа сходимости и проверки на переобучение.

In [ ]:
import matplotlib.pyplot as plt

# Графики метрик
fig, axs = plt.subplots(3, 1, figsize=(10, 15))

# Loss
axs[0].plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss', marker='o')
axs[0].plot(range(1, len(val_losses) + 1), val_losses, label='Val Loss', marker='o')
axs[0].set_title('Loss over Epochs')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].grid(True)

# PSNR
axs[1].plot(range(1, len(train_psnrs) + 1), train_psnrs, label='Train PSNR', marker='o')
axs[1].plot(range(1, len(val_psnrs) + 1), val_psnrs, label='Val PSNR', marker='o')
axs[1].set_title('PSNR over Epochs')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('PSNR (dB)')
axs[1].legend()
axs[1].grid(True)

# SSIM
axs[2].plot(range(1, len(train_ssims) + 1), train_ssims, label='Train SSIM', marker='o')
axs[2].plot(range(1, len(val_ssims) + 1), val_ssims, label='Val SSIM', marker='o')
axs[2].set_title('SSIM over Epochs')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('SSIM')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.savefig('restormer_metrics.png')
plt.show()
print("Графики метрик сохранены как restormer_metrics.png. Проверьте на переобучение (val метрики не ухудшаются).")